#<font color='#3dc1d3'>  CLINICAL TRIAL - DATA PREPERATION

#Outline
---
# <font color='#3dc1d3'>  
1. Dataset- (Diabetes)
2. Install required library, import and read Dataset
3. Data Cleaning
5. Save the cleaned dataset <br>


#1. Importing necessary libraries


In [5]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
from nltk.tokenize import RegexpTokenizer
import re
import os

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#2. Reading the Diabetes Dataset

In [7]:
data = pd.read_csv('/content/drive/MyDrive/Practicum/Final_folders/Submission/2.Data_preperation_done/clinical_trial - clinical_trial.csv')
data.head(10)

,Rank,NCT Number,Title,Status,Study Results,Conditions,Interventions,Sponsor/Collaborators,Age,Phases,Funded Bys,Study Type,Study Designs,Locations,Study Documents,URL
0,1,NCT03763474,Euglyca Application in Children and Adolescent...,Completed,No Results Available,"Diabetes Mellitus, Type 1",Combination Product: Euglyca application,Aristotle University Of Thessaloniki,"6 Years to 18 Years (Child, Adult)",Not Applicable,Other,Interventional,Allocation: Randomized|Intervention Model: Par...,Endocrine Unit of 3rd Department of Pediatrics...,"""Study Protocol and Statistical Analysis Plan""...",https://ClinicalTrials.gov/show/NCT03763474
1,2,NCT05013294,Using mHealth (Mobile Health) to Optimize Glyc...,Completed,No Results Available,Diabetes,Other: Intervention,KU Leuven|Jomo Kenyatta University of Agricult...,"Child, Adult, Older Adult",Not Applicable,Other,Interventional,Allocation: Randomized|Intervention Model: Par...,Jomo Kenyatta University of Agriculture and Te...,"""Study Protocol"", https://ClinicalTrials.gov/P...",https://ClinicalTrials.gov/show/NCT05013294
2,3,NCT03108521,SNP Study of DPP-4 and GLP-1R in Chinese Peopl...,Completed,Has Results,Diabetes Mellitus,Drug: Sitagliptin,Sichuan Provincial People's Hospital,"18 Years to 70 Years (Adult, Older Adult)",Phase 4,Other,Interventional,Allocation: Non-Randomized|Intervention Model:...,"Sichuan provincial people's hospital, Chengdu,...","""Study Protocol and Statistical Analysis Plan""...",https://ClinicalTrials.gov/show/NCT03108521
3,4,NCT02928952,Mindful Stress Reduction in Diabetes Self-mana...,Completed,Has Results,Diabetes Mellitus,Behavioral: Mind-STRIDE,VA Office of Research and Development,"18 Years and older (Adult, Older Adult)",Not Applicable,U.S. Fed,Interventional,Allocation: Randomized|Intervention Model: Par...,VA Pittsburgh Healthcare System University Dri...,"""Study Protocol and Statistical Analysis Plan""...",https://ClinicalTrials.gov/show/NCT02928952
4,5,NCT05106231,Effectiveness of PICC Improving HBA1C and Know...,Recruiting,No Results Available,Diabetes Mellitus|Type2 Diabetes,Behavioral: PICC Program|Behavioral: Control,"Clinical Research Centre, Malaysia|Ministry of...","18 Years and older (Adult, Older Adult)",Not Applicable,Other,Interventional,Allocation: Randomized|Intervention Model: Fac...,"Klinik Kesihatan Daro, Daro, Sarawak, Malaysia...","""Study Protocol, Statistical Analysis Plan, an...",https://ClinicalTrials.gov/show/NCT05106231
5,6,NCT02002130,The Use of Glutamic Acid Decarboxylase (GAD) a...,Completed,No Results Available,Type I Diabetes,Drug: Placebo GABA and Placebo GAD-alum|Drug: ...,University of Alabama at Birmingham|Diamyd Inc...,"4 Years to 18 Years (Child, Adult)",Phase 1,Other|Industry,Interventional,Allocation: Randomized|Intervention Model: Par...,"Children's of Alabama, Birmingham, Alabama, Un...","""Study Protocol and Statistical Analysis Plan""...",https://ClinicalTrials.gov/show/NCT02002130
6,7,NCT02641522,Modulation of STAT3 Signaling With Siltuximab ...,Completed,Has Results,Type 1 Diabetes,Drug: Siltuximab,"Carla Greenbaum, MD|Janssen Research & Develop...",18 Years to 45 Years (Adult),Early Phase 1,Other|Industry,Interventional,Allocation: N/A|Intervention Model: Single Gro...,"Benaroya Research Institute, Seattle, Washingt...","""Study Protocol and Statistical Analysis Plan""...",https://ClinicalTrials.gov/show/NCT02641522
7,8,NCT03224234,A Randomized Study to Evaluate the Efficacy of...,Completed,Has Results,Diabetes,Device: Insulclock with feedback|Device: Insul...,Emory University|Insulcoud S.L.,"18 Years to 80 Years (Adult, Older Adult)",Not Applicable,Other,Interventional,Allocation: Randomized|Intervention Model: Cro...,"Emory Clinic, Emory University Hospital (non-C...","""Study Protocol and Statistical Analysis Plan""...",https://ClinicalTrials.gov/show/NCT03224234
8,9,NCT03626909,Diabetes Treatment in Rural Guatemala,Completed,No Results Available,Diabetes,Other: Primary visit and assessment|Other: Fol.

#3. Data Cleaning

###Check for null values

In [8]:
data.isnull().sum()

Rank                      0
NCT Number                0
Title                     0
Status                    0
Study Results             0
Conditions                0
Interventions            25
Sponsor/Collaborators     0
Age                       0
Phases                   98
Funded Bys                0
Study Type                0
Study Designs             0
Locations                 0
Study Documents           0
URL                       0
dtype: int64

1. Null values for locations 
   --> We imputed null values for loacations with sponsors
2. Null values for intervention 
   --> Study can be either observational or Interventional. Thus the null values in interventions can be imputed
3. Null values for phases we cannot impute without evidence

##Column Transformations

### Status

*   We are choosing the Status column to be our target variable because it determines the present status of the experiment
*   It has 11 values, which we will be transform into Completed & Not Completed so that we can treat our problem as binary classification. 
*   As a result, all remaining entries except 'completed' are changed to 'not completed'
*   Status column has no missing data.







In [9]:
data['Status'].unique()

array(['Completed', 'Recruiting', 'Active, not recruiting', 'Terminated',
       'Unknown status', 'Not yet recruiting', 'Enrolling by invitation',
       'Withdrawn', 'Suspended'], dtype=object)

In [10]:
data['Status'] = data['Status'].replace({
        'Recruiting': 'Not Completed',
        'Unknown status': 'Not Completed',
        'Terminated': 'Not Completed',
        'Active, not recruiting': 'Not Completed',
        'Not yet recruiting': 'Not Completed',
        'Withdrawn': 'Not Completed',
        'Enrolling by invitation': 'Not Completed',
        'Suspended': 'Not Completed',
        'Available': 'Not Completed',
        'No longer available': 'Not Completed',
})

### Interventions

*   A process or action that is the focus of a clinical study. Interventions include drugs, medical devices, procedures, vaccines, and other products that are either investigational or already available. 
*   There are 733 missing values in the column Interventions. For these rows, the study type is Observational. Hence, we will replace NaN with 'No interventions' category. 
*   From the interventions column, we can derive a new column which is Intervention_type. 



In [11]:
data['Study Type'].unique()

array(['Interventional', 'Observational'], dtype=object)

In [12]:
interventions_na_df = data[data['Interventions'].isna()]

In [13]:
interventions_na_df['Study Type'].value_counts()
data.Interventions = data.Interventions.fillna('No Interventions')
interventions_na_df = data[data['Interventions'].isna()]
data['Intervention_type'] = data['Interventions'].str.split(':').str[0]

In [14]:
data.isnull().sum()

Rank                      0
NCT Number                0
Title                     0
Status                    0
Study Results             0
Conditions                0
Interventions             0
Sponsor/Collaborators     0
Age                       0
Phases                   98
Funded Bys                0
Study Type                0
Study Designs             0
Locations                 0
Study Documents           0
URL                       0
Intervention_type         0
dtype: int64

###Extracted pdf links from document column

In [15]:
data["Study Documents"]=data["Study Documents"].astype(str)
pattern = r'(https?:\/\/(?:www\.)?[-a-zA-Z0-9@:%._+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}[-a-zA-Z0-9()@:%_+.~#?&/=]*)' 
data['links'] = ''
data['links']= data["Study Documents"].str.extract(pattern, expand=True)

In [16]:
data= data[['NCT Number','Status','Conditions','Sponsor/Collaborators','Age','Funded Bys','Study Designs','Locations','links']]

In [17]:
data.head(2)

,NCT Number,Status,Conditions,Sponsor/Collaborators,Age,Funded Bys,Study Designs,Locations,links
0,NCT03763474,Completed,"Diabetes Mellitus, Type 1",Aristotle University Of Thessaloniki,"6 Years to 18 Years (Child, Adult)",Other,Allocation: Randomized|Intervention Model: Par...,Endocrine Unit of 3rd Department of Pediatrics...,https://ClinicalTrials.gov/ProvidedDocs/74/NCT...
1,NCT05013294,Completed,Diabetes,KU Leuven|Jomo Kenyatta University of Agricult...,"Child, Adult, Older Adult",Other,Allocation: Randomized|Intervention Model: Par...,Jomo Kenyatta University of Agriculture and Te...,https://ClinicalTrials.gov/ProvidedDocs/94/NCT...


In [18]:
data.isnull().sum()

NCT Number               0
Status                   0
Conditions               0
Sponsor/Collaborators    0
Age                      0
Funded Bys               0
Study Designs            0
Locations                0
links                    0
dtype: int64

In [19]:
data.Conditions[0]

'Diabetes Mellitus, Type 1'

In [20]:
data= data.rename(columns = {'Sponsor/Collaborators':'Sponsor','Funded Bys': 'Funded_Bys','Study Designs':'Study_Designs'})

###Text cleaning

In [21]:
#Removed single pipe and brackets
def listToStringWithoutBrackets(text):
    return str(text).replace('[',' ').replace(']',' ')

def listToStringWithoutpipe(text):
    return str(text).replace('|',' ').replace('||',' ')

In [22]:
data["Conditions"] = data.Conditions.apply(func = listToStringWithoutBrackets)
data["Conditions"] = data.Conditions.apply(func = listToStringWithoutpipe)

data["Sponsor"] = data.Sponsor.apply(func = listToStringWithoutBrackets)
data["Sponsor"] = data.Sponsor.apply(func = listToStringWithoutpipe)

data["Age"] = data.Age.apply(func = listToStringWithoutBrackets)
data["Age"] = data.Age.apply(func = listToStringWithoutpipe)

data["Funded_Bys"] = data.Funded_Bys.apply(func = listToStringWithoutBrackets)
data["Funded_Bys"] = data.Funded_Bys.apply(func = listToStringWithoutpipe)

data["Study_Designs"] = data.Study_Designs.apply(func = listToStringWithoutBrackets)
data["Study_Designs"] = data.Study_Designs.apply(func = listToStringWithoutpipe)

data["Locations"] = data.Locations.apply(func = listToStringWithoutBrackets)
data["Locations"] = data.Locations.apply(func = listToStringWithoutpipe)

In [23]:
#removed hyper links 
data["Conditions"] = data["Conditions"].apply(lambda x: re.sub(r'https?:\/\/.*?[\s+]', '', x.replace("|"," ") + " "))
data["Sponsor"] = data["Sponsor"].apply(lambda x: re.sub(r'https?:\/\/.*?[\s+]', '', x.replace("|"," ") + " "))
data["Age"] = data["Age"].apply(lambda x: re.sub(r'https?:\/\/.*?[\s+]', '', x.replace("|"," ") + " "))
data["Funded_Bys"] = data["Funded_Bys"].apply(lambda x: re.sub(r'https?:\/\/.*?[\s+]', '', x.replace("|"," ") + " "))
data["Study_Designs"] = data["Study_Designs"].apply(lambda x: re.sub(r'https?:\/\/.*?[\s+]', '', x.replace("|"," ") + " "))
data["Locations"] = data["Locations"].apply(lambda x: re.sub(r'https?:\/\/.*?[\s+]', '', x.replace("|"," ") + " "))

#Strip Punctation
data["Conditions"] = data["Conditions"].apply(lambda x: re.sub(r'[\.+]', ".",x))
data["Sponsor"] = data["Sponsor"].apply(lambda x: re.sub(r'[\.+]', ".",x))
data["Age"] = data["Age"].apply(lambda x: re.sub(r'[\.+]', ".",x))
data["Funded_Bys"] = data["Funded_Bys"].apply(lambda x: re.sub(r'[\.+]', ".",x))
data["Study_Designs"] = data["Study_Designs"].apply(lambda x: re.sub(r'[\.+]', ".",x))
data["Locations"] = data["Locations"].apply(lambda x: re.sub(r'[\.+]', ".",x))

#Remove multiple fullstops
data["Conditions"] = data["Conditions"].apply(lambda x: re.sub(r'[^\w\s]','',x))
data["Sponsor"] = data["Sponsor"].apply(lambda x: re.sub(r'[^\w\s]','',x))
data["Age"] = data["Age"].apply(lambda x: re.sub(r'[^\w\s]','',x))
data["Funded_Bys"] = data["Funded_Bys"].apply(lambda x: re.sub(r'[^\w\s]','',x))
data["Study_Designs"] = data["Study_Designs"].apply(lambda x: re.sub(r'[^\w\s]','',x))
data["Locations"] = data["Locations"].apply(lambda x: re.sub(r'[^\w\s]','',x))

#Remove Non-words
#data["Conditions"] = data["Conditions"].apply(lambda x: re.sub(r'[^a-zA-Z\s]','',x))
data["Sponsor"] = data["Sponsor"].apply(lambda x: re.sub(r'[^a-zA-Z\s]','',x))
data["Age"] = data["Age"].apply(lambda x: re.sub(r'[^a-zA-Z\s]','',x))
data["Funded_Bys"] = data["Funded_Bys"].apply(lambda x: re.sub(r'[^a-zA-Z\s]','',x))
data["Study_Designs"] = data["Study_Designs"].apply(lambda x: re.sub(r'[^a-zA-Z\s]','',x))
data["Locations"] = data["Locations"].apply(lambda x: re.sub(r'[^a-zA-Z\s]','',x))

#Convert posts to lowercase
data["Conditions"] = data["Conditions"].apply(lambda x: x.lower())
data["Sponsor"] = data["Sponsor"].apply(lambda x: x.lower())
data["Age"] = data["Age"].apply(lambda x: x.lower())
data["Funded_Bys"] = data["Funded_Bys"].apply(lambda x: x.lower())
data["Study_Designs"] = data["Study_Designs"].apply(lambda x: x.lower())
data["Locations"] = data["Locations"].apply(lambda x: x.lower())

#Remove multiple letter repeating words
data["Conditions"] = data["Conditions"].apply(lambda x: re.sub(r'([a-z])\1{2,}[\s|\w]*','',x)) 
data["Sponsor"] = data["Sponsor"].apply(lambda x: re.sub(r'([a-z])\1{2,}[\s|\w]*','',x)) 
data["Age"] = data["Age"].apply(lambda x: re.sub(r'([a-z])\1{2,}[\s|\w]*','',x)) 
data["Funded_Bys"] = data["Funded_Bys"].apply(lambda x: re.sub(r'([a-z])\1{2,}[\s|\w]*','',x)) 
data["Study_Designs"] = data["Study_Designs"].apply(lambda x: re.sub(r'([a-z])\1{2,}[\s|\w]*','',x)) 
data["Locations"] = data["Locations"].apply(lambda x: re.sub(r'([a-z])\1{2,}[\s|\w]*','',x)) 

In [24]:
data.isnull().sum()

NCT Number       0
Status           0
Conditions       0
Sponsor          0
Age              0
Funded_Bys       0
Study_Designs    0
Locations        0
links            0
dtype: int64

In [25]:
data.head(5)

,NCT Number,Status,Conditions,Sponsor,Age,Funded_Bys,Study_Designs,Locations,links
0,NCT03763474,Completed,diabetes mellitus type 1,aristotle university of thessaloniki,years to years child adult,other,allocation randomized intervention model paral...,endocrine unit of rd department of pediatrics ...,https://ClinicalTrials.gov/ProvidedDocs/74/NCT...
1,NCT05013294,Completed,diabetes,ku leuven jomo kenyatta university of agricult...,child adult older adult,other,allocation randomized intervention model paral...,jomo kenyatta university of agriculture and te...,https://ClinicalTrials.gov/ProvidedDocs/94/NCT...
2,NCT03108521,Completed,diabetes mellitus,sichuan provincial peoples hospital,years to years adult older adult,other,allocation nonrandomized intervention model pa...,sichuan provincial peoples hospital chengdu si...,https://ClinicalTrials.gov/ProvidedDocs/21/NCT...
3,NCT02928952,Completed,diabetes mellitus,va office of research and development,years and older adult older adult,us fed,allocation randomized intervention model paral...,va pittsburgh healthcare system university dri...,https://ClinicalTrials.gov/ProvidedDocs/52/NCT...
4,NCT05106231,Not Completed,diabetes mellitus type2 diabetes,clinical research centre malaysia ministry of ...,years and older adult older adult,other,allocation randomized intervention model facto...,klinik kesihatan daro daro sarawak malaysia kl...,https://ClinicalTrials.gov/ProvidedDocs/31/NCT...


#5. Save the transformed file

In [27]:
os.chdir('/content/drive/MyDrive/Practicum/Final_folders/Submission/2.Data_preperation_done/')
data.to_csv('data_cleaned_clinical_trial.csv')
